In [2]:
print("Hello")

Hello


In [4]:
import pandas as pd
import os


# Directory where your CSV files are located
folder_path = './summarized_patents_claims/'


# List to hold data from each CSV file
data_frames = []


# Loop through all the CSV files in the directory
for file_name in os.listdir(folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(folder_path, file_name)
        # Read the CSV file and append it to the list of data frames
        df = pd.read_csv(file_path)
        data_frames.append(df)


# Concatenate all the data frames into one
merged_df = pd.concat(data_frames, ignore_index=True)


# Save the merged data frame to a new CSV file
output_file_path = './test_summarization_loop/merged_file_claims.csv'
merged_df.to_csv(output_file_path, index=False)


print(f'Merged CSV saved to {output_file_path}')


Merged CSV saved to ./test_summarization_loop/merged_file_claims.csv


In [5]:
import pandas as pd
data_patent = pd.read_csv('./test_summarization_loop/merged_file_claims.csv')

In [9]:
rows = len(data_patent.axes[0])
cols = len(data_patent.axes[1])
print(rows, cols)
display(data_patent)

3720 11


,Unnamed: 0,Country,PatentNumber,ApplicationReference,Kind,Date,Abstract,OrganizationName,SummaryOfInvention,Claims,SummaryofPatentClaims
0,938,US,RE049823,reissue,E1,2024-01-30,Apparatuses and methods for transmitting and r...,"Samsung Electronics Co., Ltd.",{'h-0001 - PRIORITY': ['This application claim...,['1. A method for transmitting a signal by a b...,Summary: The proposed research aims to improve...
1,1336,US,11883166,utility,B2,2024-01-30,An apparatus for estimating a component of an ...,"SAMSUNG ELECTRONICS CO., LTD.",{'h-0001 - CROSS-REFERENCE TO RELATED APPLICAT...,['1. An apparatus for estimating a component o...,Summary: The research aims to improve the accu...
2,1342,US,11883172,utility,B2,2024-01-30,An electronic device may determine whether a u...,"Samsung Electronics Co., Ltd.",{'h-0001 - CROSS-REFERENCE TO RELATED APPLICAT...,['1. An electronic device comprising: a batter...,Summary:\nThe present invention aims to improv...
3,2008,US,11883795,utility,B2,2024-01-30,Provided are an ammonia decomposition catalyst...,"SAMSUNG ENGINEERING CO., LTD.",{'h-0001 - CROSS-REFERENCE TO RELATED APPLICAT...,['1. An ammonia decomposition catalyst compris...,Summary:\nThe research aims to improve the eff...
4,2118,US,11883905,utility,B2,2024-01-30,A wafer processing apparatus includes: a laser...,"Samsung Electronics Co., Ltd.",{'h-0001 - CROSS-REFERENCE TO RELATED APPLICAT...,['1. A wafer processing apparatus comprising: ...,Summary:\nThe research aims to improve the ene...
...,...,...,...,...,...,...,...,...,...,...,...
3715,7672,US,11944003,utility,B2,2024-03-26,An organic light-emitting device having a low ...,"Samsung Display Co., Ltd.",{'h-0001 - CROSS-REFERENCE TO RELATED APPLICAT...,['1. An organic light-emitting device comprisi...,"rd,ator,ator,ot,ot,ot,otener,rapenerator,rapen..."
3716,7674,US,11944005,utility,B2,2024-03-26,"The invention relates to an organic molecule, ...","SAMSUNG DISPLAY CO., LTD.",{'Initial Paragraphs': ['The invention relates...,"['1. An organic molecule, comprising Formula A...",Summary: The research aims to improve the ener...
3717,7675,US,11944006,utility,B2,2024-03-26,A diamine compound includes a naphthyl-phenyl ...,"Samsung Display Co., Ltd.",{'h-0001 - CROSS-REFERENCE TO RELATED APPLICAT...,['1. An organic light-emitting device comprisi...,Summary: The research aims to improve the ener...
3718,7677,US,11944008,utility,B2,2024-03-26,Provided are a heterocyclic compound represent...,"Samsung Display Co., Ltd.",{'h-0001 - CROSS-REFERENCE TO RELATED APPLICAT...,['1. An organic light-emitting device comprisi...,The research aims to improve the energy effici...


In [10]:
patent_data = data_patent[["PatentNumber","ApplicationReference","Date","OrganizationName","SummaryofPatentClaims"]]

In [11]:
rows = len(patent_data.axes[0])
cols = len(patent_data.axes[1])
print(rows, cols)

3720 5


In [12]:
patent_data.to_csv('./test_summarization_loop/patents_summarized_claims_clean.csv', index=False)

In [14]:
from langchain_community.document_loaders.csv_loader import CSVLoader
import sys
import csv

csv.field_size_limit(sys.maxsize)

131072

In [15]:
loader = CSVLoader(file_path='./test_summarization_loop/patents_summarized_claims_clean.csv')

documents = loader.load()

In [16]:
print(type(documents))

<class 'list'>


In [17]:
documents[0].page_content

'PatentNumber: RE049823\nApplicationReference: reissue\nDate: 2024-01-30\nOrganizationName: Samsung Electronics Co., Ltd.\nSummaryofPatentClaims: Summary: The proposed research aims to improve the efficiency of a base station in a mobile communication system by multiplexing common control channel signals and RU specific data signals for different radio units (RUs) through a common public radio interface (CPRI) based on a wired medium. The method determines the UEs with a maximum scheduling metric based on channel quality information and generates a common control channel signal for the UEs. The RU specific data signals are generated for each RU, and the signals are multiplexed and transmitted to the RUs through the CPRI. The proposed method aims to reduce the number of signals transmitted by the base station, thereby improving the energy efficiency of the base station. However, the proposed method may require additional signal processing and transmission resources, which could potentia

In [18]:
len(documents)

3720

In [19]:
#Embedding model load

from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import Chroma
from codecarbon import EmissionsTracker
import time

print("Start of code time: ",time.time())

tracker = EmissionsTracker()
tracker.start()

model_name = "BAAI/bge-large-en-v1.5"
# model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    # model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
    # ,
    # query_instruction="为这个句子生成表示以用于检索相关文章："
)


emissions: float = tracker.stop()
print("Emissions for this code execution is: ", emissions)
print("End of code time: ",time.time())

[codecarbon WARNING @ 20:42:17] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 20:42:17] [setup] RAM Tracking...
[codecarbon INFO @ 20:42:17] [setup] GPU Tracking...
[codecarbon INFO @ 20:42:17] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 20:42:17] [setup] CPU Tracking...
[codecarbon WARNING @ 20:42:17] No CPU tracking mode found. Falling back on CPU constant mode.


Start of code time:  1717962137.5860078


[codecarbon WARNING @ 20:42:18] We saw that you have a Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 20:42:18] CPU Model on constant consumption mode: Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz
[codecarbon INFO @ 20:42:18] >>> Tracker's metadata:
[codecarbon INFO @ 20:42:18]   Platform system: Linux-5.4.0-182-generic-x86_64-with-glibc2.31
[codecarbon INFO @ 20:42:18]   Python version: 3.12.1
[codecarbon INFO @ 20:42:18]   CodeCarbon version: 2.4.1
[codecarbon INFO @ 20:42:18]   Available RAM : 503.339 GB
[codecarbon INFO @ 20:42:18]   CPU count: 112
[codecarbon INFO @ 20:42:18]   CPU model: Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz
[codecarbon INFO @ 20:42:18]   GPU count: 2
[codecarbon INFO @ 20:42:18]   GPU model: 2 x NVIDIA A40
/home/c7361293/miniconda3/envs/triztest/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/u

Emissions for this code execution is:  0.0003082000930634631
End of code time:  1717962153.243683


In [21]:
print(model)

client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
) model_name='BAAI/bge-large-en-v1.5' cache_folder=None model_kwargs={} encode_kwargs={'normalize_embeddings': True} query_instruction='Represent this question for searching relevant passages: ' embed_instruction=''


In [22]:
import os # Importing os module for operating system functionalities
import shutil # Importing shutil module for high-level file operations

In [23]:
 # Clear out the existing database directory if it exists
if os.path.exists("./db_claims"):
    shutil.rmtree("./db_claims")


In [26]:
#Chroma DB saving files and embeddings

from langchain.vectorstores import Chroma
from codecarbon import EmissionsTracker
import time

print("Start of code time: ",time.time())

tracker = EmissionsTracker()
tracker.start()

chroma_db = Chroma.from_documents(
    documents, model, persist_directory="./db_claims"
)

chroma_db.persist()

emissions: float = tracker.stop()
print("Emissions for this code execution is: ", emissions)
print("End of code time: ",time.time())


[codecarbon WARNING @ 20:43:52] Invalid gpu_ids format. Expected a string or a list of ints.
[codecarbon INFO @ 20:43:52] [setup] RAM Tracking...
[codecarbon INFO @ 20:43:52] [setup] GPU Tracking...
[codecarbon INFO @ 20:43:52] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 20:43:52] [setup] CPU Tracking...
[codecarbon WARNING @ 20:43:52] No CPU tracking mode found. Falling back on CPU constant mode.


Start of code time:  1717962232.87518


[codecarbon WARNING @ 20:43:54] We saw that you have a Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 20:43:54] CPU Model on constant consumption mode: Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz
[codecarbon INFO @ 20:43:54] >>> Tracker's metadata:
[codecarbon INFO @ 20:43:54]   Platform system: Linux-5.4.0-182-generic-x86_64-with-glibc2.31
[codecarbon INFO @ 20:43:54]   Python version: 3.12.1
[codecarbon INFO @ 20:43:54]   CodeCarbon version: 2.4.1
[codecarbon INFO @ 20:43:54]   Available RAM : 503.339 GB
[codecarbon INFO @ 20:43:54]   CPU count: 112
[codecarbon INFO @ 20:43:54]   CPU model: Intel(R) Xeon(R) Gold 6330 CPU @ 2.00GHz
[codecarbon INFO @ 20:43:54]   GPU count: 2
[codecarbon INFO @ 20:43:54]   GPU model: 2 x NVIDIA A40
[codecarbon INFO @ 20:44:12] Energy consumed for RAM : 0.000787 kWh. RAM Power : 188.75217247009277 W
[codecarbon INFO @ 20:44:12] Energy consumed for all GPUs : 0.001416 kWh. Total GPU Power : 339.363952716

Emissions for this code execution is:  0.001628985280054739
End of code time:  1717962277.0582168


In [28]:
vectordb = Chroma(persist_directory="./db_claims", embedding_function= model)
# retriever = vectordb.as_retriever(search_kwargs = {"k": 10})
retriever = vectordb.as_retriever(search_type = "similarity_score_threshold",search_kwargs = {"k": 10, "score_threshold":0.50})
print(retriever.search_type, retriever.search_kwargs)

similarity_score_threshold {'k': 10, 'score_threshold': 0.5}


similarity {'k': 10}


In [51]:
retirve_result = retriever.get_relevant_documents("Energy Efficient and Sustainable Touch Screen Display")

In [52]:
len(retirve_result)

10

In [53]:
retirve_result[0]

Document(page_content='PatentNumber: 11910672\nApplicationReference: utility\nDate: 2024-02-20\nOrganizationName: Samsung Display Co., Ltd.\nSummaryofPatentClaims: Summary: The research aims to improve the energy efficiency of display devices by reducing the number of power supply lines and increasing the capacitance of the capacitor electrode. The proposed method can reduce the power consumption of the touch panel by 20%, but may have some limitations such as reduced display quality due to the reduced number of power supply lines. Despite these limitations, the proposed method has the potential to significantly improve the energy efficiency of display devices, making them more environmentally friendly and cost-effective.', metadata={'row': 3112, 'source': './test_summarization_loop/patents_summarized_claims_clean.csv'})

In [54]:
import re

pattern = re.compile(r"\nSummaryofPatentClaims: Summary:\n(.*)", re.DOTALL)

match = pattern.search(retirve_result[0].page_content)

if match:
    print(match.group(1).strip())

In [57]:
retirve_result

[Document(page_content='PatentNumber: 11910672\nApplicationReference: utility\nDate: 2024-02-20\nOrganizationName: Samsung Display Co., Ltd.\nSummaryofPatentClaims: Summary: The research aims to improve the energy efficiency of display devices by reducing the number of power supply lines and increasing the capacitance of the capacitor electrode. The proposed method can reduce the power consumption of the touch panel by 20%, but may have some limitations such as reduced display quality due to the reduced number of power supply lines. Despite these limitations, the proposed method has the potential to significantly improve the energy efficiency of display devices, making them more environmentally friendly and cost-effective.', metadata={'row': 3112, 'source': './test_summarization_loop/patents_summarized_claims_clean.csv'}),
 Document(page_content='PatentNumber: 11861131\nApplicationReference: utility\nDate: 2024-01-02\nOrganizationName: SAMSUNG DISPLAY CO., LTD.\nSummaryofPatentClaims: 

In [36]:
# filtered_results = [doc for score, doc in retirve_result['SummaryofPatent'] if score >= "0.7"]
# print(filtered_results)

In [36]:
type(retirve_result)

list

In [56]:
import re

for item in retirve_result:
    pattern = re.compile(r"\nSummaryofPatentClaims: Summary:\n(.*)", re.DOTALL)

    match = pattern.search(item.page_content)

    if match:
        print(match.group(1).strip())

The research aims to improve the energy efficiency of electronic devices by developing a new sensor layer design that reduces the number of sensing electrodes required for touch detection while maintaining accuracy. The proposed design includes a sensing electrode with multiple portions, each extending between pixel groups, which can reduce the energy consumption of the electronic device. However, the design may face challenges in terms of manufacturing complexity and cost. Despite these challenges, the proposed design has the potential to improve the energy efficiency of electronic devices and provide a more sustainable solution for touch-sensitive displays.
The research aims to improve energy efficiency by reducing the power consumption of the display panel while maintaining display quality. The proposed solution divides the display area into three non-sensing areas and one sensing area, using a sensing electrode array to sense the touch position. The sensing electrode array is divid

In [47]:
import re

# Assuming the Document class has an attribute 'page_content' that stores the content
class Document:
    def __init__(self, page_content):
        self.page_content = page_content

for item in retirve_result:
    # Regular expression to capture Patent Number and Summary
    pattern = re.compile(r"PatentNumber: (\d+)\n.*\nSummaryofPatentClaims: Summary:\n(.*)", re.DOTALL)

    match = pattern.search(item.page_content)  # Use dot notation to access the page_content attribute

    if match:
        patent_number = match.group(1).strip()
        summary = match.group(2).strip()
        print("Patent Number:", patent_number)
        print("Summary:", summary)


Patent Number: 11861126
Summary: The research aims to improve the energy efficiency of electronic devices by developing a new sensor layer design that reduces the number of sensing electrodes required for touch detection while maintaining accuracy. The proposed design includes a sensing electrode with multiple portions, each extending between pixel groups, which can reduce the energy consumption of the electronic device. However, the design may face challenges in terms of manufacturing complexity and cost. Despite these challenges, the proposed design has the potential to improve the energy efficiency of electronic devices and provide a more sustainable solution for touch-sensitive displays.
Patent Number: 11966535
Summary: The research aims to improve energy efficiency by reducing the power consumption of the display panel while maintaining display quality. The proposed solution divides the display area into three non-sensing areas and one sensing area, using a sensing electrode array

In [49]:
from IPython.display import display, HTML

style = """
<style>
    .research-summary {
        border: 1px solid #ddd;
        border-radius: 8px;
        padding: 15px;
        margin: 10px 0;
        background-color: #f9f9f9;
        color: black;
    }
    .research-summary h3 {
        color: #336;
    }
    .research-summary p {
        text-indent: 20px;
    }
</style>
"""

import re

# Assuming the Document class has an attribute 'page_content' that stores the content
class Document:
    def __init__(self, page_content):
        self.page_content = page_content

for item in retirve_result:
    # Regular expression to capture Patent Number and Summary
    pattern = re.compile(r"PatentNumber: (\d+)\n.*\nSummaryofPatentClaims: Summary:\n(.*)", re.DOTALL)

    match = pattern.search(item.page_content)  # Use dot notation to access the page_content attribute

    if match:
        contents = ""
        patent_number = match.group(1).strip()
        summary = match.group(2).strip()
        # print("Patent Number:", patent_number)
        # print("Summary:", summary)
        contents += f"""
            <div class="research-summary">
                <h3>Patent Number:{patent_number}</h3>
                <p>{summary}</p>
            </div>
            """
        display(HTML(style + contents))


